In [2]:
import numpy as np
import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint
import pickle
from sklearn.preprocessing import StandardScaler
from keras.utils.vis_utils import plot_model

from keras.layers import MaxPooling1D,Flatten,Reshape,Bidirectional, Concatenate, Permute, Dot, Input, CuDNNLSTM, Conv1D, Multiply,Dropout,BatchNormalization
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
from keras.callbacks import ModelCheckpoint

In [3]:
address='D:\\data_mfcc\\'

In [4]:
def loadData(id):
    with open(address + f'y_train_{id}','rb') as f:
        y_train=np.array(pickle.load(f))
    with open(address+f'y_test_{id}','rb') as f:
        y_test=np.array(pickle.load(f))
    with open(address+ f'X_train_{id}','rb') as f:
        X_train = np.array(pickle.load(f))
    with open(address+f'X_test_{id}','rb') as f:
        X_test = np.array(pickle.load(f))

    scalers = {}
    for k in range(X_train.shape[2]):
        scalers[k] = StandardScaler()
        X_train[:, k, :] = scalers[k].fit_transform(X_train[:, k, :]) 

    for j in range(X_test.shape[2]):
        X_test[:, j, :] = scalers[j].transform(X_test[:, j, :])
    
    print(f'id : {id}')
    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.fit_transform(y_test)

   
    return X_train,y_train,X_test,y_test


In [ ]:
total_num_people=[]
for i in range(1,92):
    if i <10:
        total_num_people.append('100'+str(i))
    else:
        total_num_people.append('10'+str(i))

for i in total_num_people[::-1]:
    if int(i)<1028:
        X_train=[]
        X_test=[]
        y_train=[]
        y_test=[]
        i=1042
        X_train,y_train,X_test,y_test=loadData(i)
        
    else:
        continue
    

    #load data here
    #input_shape = batch_size,time_steps,input_dim

    #batch_size = X_train.shape[0]   #automatically taken no need to specify
    time_steps = X_train.shape[1]
    input_dim = X_train.shape[2]



    hid_pre_acts = [16]
    hid_post_acts = [96]
    #dense1s = [16,32,64,96,128,256]
    #dense2s = [16,32,64,96,128,256]
    for hid_pre_act in hid_pre_acts:
        for hid_post_act in hid_post_acts:
            # Defined shared layers as global variables
            repeator = RepeatVector(time_steps) #500
            concatenator = Concatenate(axis=-1)
            densor = Dense(1, activation = "relu")
            activator = Activation('softmax', name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
            dotor = Dot(axes = 1)

            #one step attention

            def one_step_attention(a,s_prev):

                s_prev = repeator(s_prev)
                concat = concatenator([a,s_prev])
                e = densor(concat)
                alphas = activator(e)
                K.print_tensor(alphas)
                context = dotor([alphas, a])

                return context
            
            post_activation_lstm_cell = CuDNNLSTM(hid_post_act,return_state=True)
            #after_post_lstm  = CuDNNLSTM(16) 
            dense_softmax = Dense(6,activation='softmax')
            conv1d1 = Conv1D(filters=96,kernel_size=2,strides=1,input_shape=(96,1),activation='relu')
            reshape = Reshape((96,1),input_shape=(96,))
            flatten = Flatten()
            #dropout = Dropout(0.4)
                # model -----------------------------------------------------------

            def model(time_steps,input_dim,hid_post_act,hid_pre_act):

                X = Input(shape =(time_steps,input_dim))
                s0 = Input(shape=(hid_post_act,),name='s0')
                c0 = Input(shape=(hid_post_act,),name='c0')
                s=s0
                c=c0
                outputs=[]


                # Pre-LSTM 16
                a = Bidirectional(CuDNNLSTM(hid_pre_act,return_sequences=True,))(X)
                a = Dropout(0.4)(a)
                a = BatchNormalization()(a)
                #6 is final output size
                for t in range(1):
                    context =one_step_attention(a,s)
                    s, _, c = post_activation_lstm_cell(context, initial_state=[s, c])
                    s = reshape(s)
                    s = batchnorm(s)
                    s = conv1d1(s)
                    s = flatten(s)
                    s=dense_softmax(s)
                    outputs.append(s)


                model = Model([X, s0, c0], outputs)

                return model       

            model = model(time_steps,input_dim,hid_post_act,hid_pre_act)

            opt = Adam()
            model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
            #model.summary()
            plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
            print(X_train.shape)
            s0 = np.zeros((X_train.shape[0], hid_post_act))
            c0 = np.zeros((X_train.shape[0], hid_post_act))
            s0_t = np.zeros((X_test.shape[0], hid_post_act))
            c0_t = np.zeros((X_test.shape[0], hid_post_act))
            print(f'id : {i}')
            filePath =f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\saved_mfcc_models\\{i}.h5'
            checkpoint = ModelCheckpoint(filepath=filePath,monitor='val_acc',mode='max',save_best_only=True,verbose=1,save_weights_only=False)
            model.fit([X_train, s0, c0], y_train, epochs=150,validation_data=([X_test,s0_t,c0_t],y_test),callbacks=[checkpoint]) 
 

id : 1042
(7360, 499, 13)
id : 1042
Train on 7360 samples, validate on 82 samples
Epoch 1/150
7360/7360 [==============================] - 34s 5ms/step - loss: 1.5491 - acc: 0.3583 - val_loss: 1.7241 - val_acc: 0.2439

Epoch 00001: val_acc improved from -inf to 0.24390, saving model to C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_mfcc_models\1042.h5


C:\Users\djaym7\Anaconda3\lib\site-packages\keras\engine\network.py:872: UserWarning: Layer cu_dnnlstm_9 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 's0_4:0' shape=(?, 96) dtype=float32>, <tf.Tensor 'c0_4:0' shape=(?, 96) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.4477 - acc: 0.4060 - val_loss: 1.6732 - val_acc: 0.2439

Epoch 00002: val_acc did not improve from 0.24390
Epoch 3/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.4143 - acc: 0.4279 - val_loss: 1.7804 - val_acc: 0.3659

Epoch 00003: val_acc improved from 0.24390 to 0.36585, saving model to C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_mfcc_models\1042.h5
Epoch 4/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.3868 - acc: 0.4421 - val_loss: 1.7520 - val_acc: 0.2683

Epoch 00004: val_acc did not improve from 0.36585
Epoch 5/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.3650 - acc: 0.4529 - val_loss: 1.6196 - val_acc: 0.3659

Epoch 00005: val_acc did not improve from 0.36585
Epoch 6/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.3507 - acc: 0.4658 - val_loss: 1.7586 - val_acc: 0.3049

Epoch 00006: val_a

7360/7360 [==============================] - 29s 4ms/step - loss: 1.1397 - acc: 0.5549 - val_loss: 1.5128 - val_acc: 0.3537

Epoch 00043: val_acc did not improve from 0.41463
Epoch 44/150
7360/7360 [==============================] - 29s 4ms/step - loss: 1.1389 - acc: 0.5580 - val_loss: 1.4666 - val_acc: 0.3537

Epoch 00044: val_acc did not improve from 0.41463
Epoch 45/150
7360/7360 [==============================] - 29s 4ms/step - loss: 1.1381 - acc: 0.5550 - val_loss: 1.5288 - val_acc: 0.3902

Epoch 00045: val_acc did not improve from 0.41463
Epoch 46/150
7360/7360 [==============================] - 29s 4ms/step - loss: 1.1493 - acc: 0.5492 - val_loss: 1.5284 - val_acc: 0.4024

Epoch 00046: val_acc did not improve from 0.41463
Epoch 47/150
7360/7360 [==============================] - 29s 4ms/step - loss: 1.1417 - acc: 0.5549 - val_loss: 1.5197 - val_acc: 0.3049

Epoch 00047: val_acc did not improve from 0.41463
Epoch 48/150
7360/7360 [==============================] - 29s 4ms/step - 


Epoch 00084: val_acc did not improve from 0.47561
Epoch 85/150
7360/7360 [==============================] - 29s 4ms/step - loss: 1.0964 - acc: 0.5728 - val_loss: 1.5460 - val_acc: 0.3659

Epoch 00085: val_acc did not improve from 0.47561
Epoch 86/150
7360/7360 [==============================] - 29s 4ms/step - loss: 1.0967 - acc: 0.5731 - val_loss: 1.6047 - val_acc: 0.3171

Epoch 00086: val_acc did not improve from 0.47561
Epoch 87/150
7360/7360 [==============================] - 29s 4ms/step - loss: 1.1045 - acc: 0.5751 - val_loss: 1.4280 - val_acc: 0.4390

Epoch 00087: val_acc did not improve from 0.47561
Epoch 88/150
7360/7360 [==============================] - 29s 4ms/step - loss: 1.0938 - acc: 0.5716 - val_loss: 1.5434 - val_acc: 0.3780

Epoch 00088: val_acc did not improve from 0.47561
Epoch 89/150
7360/7360 [==============================] - 29s 4ms/step - loss: 1.0802 - acc: 0.5747 - val_loss: 1.3839 - val_acc: 0.4024

Epoch 00089: val_acc did not improve from 0.47561
Epoch 90/

...

C:\Users\djaym7\Anaconda3\lib\site-packages\keras\engine\network.py:872: UserWarning: Layer cu_dnnlstm_11 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 's0_5:0' shape=(?, 96) dtype=float32>, <tf.Tensor 'c0_5:0' shape=(?, 96) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.4208 - acc: 0.4289 - val_loss: 1.7619 - val_acc: 0.3171

Epoch 00002: val_acc improved from 0.23171 to 0.31707, saving model to C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_mfcc_models\1042.h5
Epoch 3/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.3915 - acc: 0.4405 - val_loss: 1.6736 - val_acc: 0.3537

Epoch 00003: val_acc improved from 0.31707 to 0.35366, saving model to C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_mfcc_models\1042.h5
Epoch 4/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.3592 - acc: 0.4461 - val_loss: 1.6992 - val_acc: 0.3171

Epoch 00004: val_acc did not improve from 0.35366
Epoch 5/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.3463 - acc: 0.4625 - val_loss: 1.6610 - val_acc: 0.3537

Epoch 00005: val_acc did not improve from 0.35366
Epoch 6/150
7360/7360 [==============================] - 2

7360/7360 [==============================] - 28s 4ms/step - loss: 1.1332 - acc: 0.5539 - val_loss: 1.5817 - val_acc: 0.3293

Epoch 00043: val_acc did not improve from 0.46341
Epoch 44/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.1439 - acc: 0.5526 - val_loss: 1.5039 - val_acc: 0.3902

Epoch 00044: val_acc did not improve from 0.46341
Epoch 45/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.1313 - acc: 0.5554 - val_loss: 1.4898 - val_acc: 0.4024

Epoch 00045: val_acc did not improve from 0.46341
Epoch 46/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.1465 - acc: 0.5535 - val_loss: 1.5643 - val_acc: 0.3049

Epoch 00046: val_acc did not improve from 0.46341
Epoch 47/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.1276 - acc: 0.5615 - val_loss: 1.5997 - val_acc: 0.3902

Epoch 00047: val_acc did not improve from 0.46341
Epoch 48/150
7360/7360 [==============================] - 28s 4ms/step - 

7360/7360 [==============================] - 28s 4ms/step - loss: 1.0869 - acc: 0.5755 - val_loss: 1.4547 - val_acc: 0.3902

Epoch 00086: val_acc did not improve from 0.47561
Epoch 87/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.1055 - acc: 0.5687 - val_loss: 1.5937 - val_acc: 0.3293

Epoch 00087: val_acc did not improve from 0.47561
Epoch 88/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.0845 - acc: 0.5823 - val_loss: 1.5876 - val_acc: 0.3659

Epoch 00088: val_acc did not improve from 0.47561
Epoch 89/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.0963 - acc: 0.5716 - val_loss: 1.6292 - val_acc: 0.3171

Epoch 00089: val_acc did not improve from 0.47561
Epoch 90/150
7360/7360 [==============================] - 28s 4ms/step - loss: 1.0890 - acc: 0.5779 - val_loss: 1.7056 - val_acc: 0.3049

Epoch 00090: val_acc did not improve from 0.47561
Epoch 91/150
7360/7360 [==============================] - 28s 4ms/step - 

  96/7360 [..............................] - ETA: 28s - loss: 1.1962 - acc: 0.5208